# KWS Training Task

blabla

## 0. Install software

The following steps should ideally done before launching this Jupyter notebook!

In [10]:
# TODO

## 1. Python imports

Python builtin dependencies

In [1]:
import argparse
from pathlib import Path

Third party dependencies

In [2]:
import tensorflow as tf
import numpy as np

2022-03-15 16:30:24.573611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 16:30:24.573626: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Helper scripts

In [7]:
import data
import models
from test_tflite import tflite_test

## 2. Define training parameters

In [ ]:
FLAGS = argparse.Namespace()  # TODO: use upper-case constants instead?

# TODO: useful?
FLAGS.model_name = "kws_model"

# Location of speech training data archive on the web.
FLAGS.data_url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'

# Where to download the speech training data to.
FLAGS.data_dir = '/tmp/speech_dataset/'

# How loud the background noise should be, between 0 and 1.
FLAGS.background_volume = 0.1

# How many of the training samples have background noise mixed in.
FLAGS.background_frequency = 0.8

# How much of the training data should be silence.
FLAGS.silence_percentage = 10.0

# How much of the training data should be unknown words
FLAGS.unknown_percentage = 10.0

# Range to randomly shift the training audio by in time.
FLAGS.time_shift_ms = 100.0

# What percentage of wavs to use as a test set.
FLAGS.testing_percentage = 10

# What percentage of wavs to use as a validation set.
FLAGS.validation_percentage = 10

# Expected sample rate of the wavs
FLAGS.sample_rate = 16000

# Expected duration in milliseconds of the wavs
FLAGS.clip_duration_ms = 1000

# How long each spectrogram timeslice is
FLAGS.window_size_ms = 30.0

# How long each spectrogram timeslice is
FLAGS.window_stride_ms = 10.0

# How many bins to use for the MFCC fingerprint
FLAGS.dct_coefficient_count = 40

# How many training loops to run
FLAGS.how_many_training_steps = '1500,300'

# How often to evaluate the training results.
FLAGS.eval_step_interval = 400

# How large a learning rate to use when training.
FLAGS.learning_rate = '0.001,0.0001'

# How many items to train with at once
FLAGS.batch_size = 100

# Where to save summary logs for TensorBoard.
FLAGS.summaries_dir = '/tmp/retrain_logs'

# Words to use (others will be added to an unknown label)
FLAGS.wanted_words = 'yes,no,up,down,left,right,on,off,stop,go'

# Directory to write event logs and checkpoint.
FLAGS.train_dir = '/tmp/speech_commands_train'

## 3. Create Keras Model

Define a model architecture

In [4]:
def create_model(model_settings):
    """Builds a model with a single depthwise-convolution layer followed by a single fully-connected layer.
    For details see https://arxiv.org/abs/1711.07128.
    Args:
        model_settings: Dict of different settings for model training.
    Returns:
        tf.keras Model of the 'CNN' architecture.
    """

    # Get relevant model setting.
    input_frequency_size = model_settings['dct_coefficient_count']
    input_time_size = model_settings['spectrogram_length']

    ### Task X: REPLACE CODE BELOW ###

    inputs = tf.keras.Input(shape=(model_settings["fingerprint_size"]), name="input")

    # Reshape the flattened input.
    x = tf.reshape(inputs, shape=(-1, input_time_size, input_frequency_size, 1))

    # First convolution.
    x = tf.keras.layers.DepthwiseConv2D(
        depth_multiplier=8,
        kernel_size=(10, 8),
        strides=(2, 2),
        padding="SAME",
        activation="relu",
    )(x)

    # Flatten for fully connected layers.
    x = tf.keras.layers.Flatten()(x)

    # Output fully connected.
    output = tf.keras.layers.Dense(units=model_settings["label_count"], activation="softmax")(x)
    
    ### Task X: REPLACE CODE ABOVE ###

    return tf.keras.Model(inputs, output)

## 4. Run Training

Define training procedure

In [ ]:
def train():
    model_settings = models.prepare_model_settings(len(data.prepare_words_list(FLAGS.wanted_words.split(','))),
                                                   FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
                                                   FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)

    # Create the model.
    model = create_model(model_settings)

    audio_processor = data.AudioProcessor(data_url=FLAGS.data_url,
                                          data_dir=FLAGS.data_dir,
                                          silence_percentage=FLAGS.silence_percentage,
                                          unknown_percentage=FLAGS.unknown_percentage,
                                          wanted_words=FLAGS.wanted_words.split(','),
                                          validation_percentage=FLAGS.validation_percentage,
                                          testing_percentage=FLAGS.testing_percentage,
                                          model_settings=model_settings)

    # We decay learning rate in a constant piecewise way to help learning.
    training_steps_list = list(map(int, FLAGS.how_many_training_steps.split(',')))
    learning_rates_list = list(map(float, FLAGS.learning_rate.split(',')))
    lr_boundary_list = training_steps_list[:-1]  # Only need the values at which to change lr.
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundary_list,
                                                                       values=learning_rates_list)

    # Specify the optimizer configurations.
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

	# Compile the model.
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Prepare/split the dataset.
    train_data = audio_processor.get_data(audio_processor.Modes.TRAINING,
                                          FLAGS.background_frequency, FLAGS.background_volume,
                                          int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000))
    train_data = train_data.repeat().batch(FLAGS.batch_size).prefetch(tf.data.AUTOTUNE)
    val_data = audio_processor.get_data(audio_processor.Modes.VALIDATION)
    val_data = val_data.batch(FLAGS.batch_size).prefetch(tf.data.AUTOTUNE)

    # We train for a max number of iterations so need to calculate how many 'epochs' this will be.
    training_steps_max = np.sum(training_steps_list)
    training_epoch_max = int(np.ceil(training_steps_max / FLAGS.eval_step_interval))

    # Callbacks.
    train_dir = Path(FLAGS.train_dir) / "best"
    train_dir.mkdir(parents=True, exist_ok=True)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=(train_dir / (FLAGS.model_name + "_{val_accuracy:.3f}_ckpt")),
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=FLAGS.summaries_dir)

    # Train the model.
    model.fit(x=train_data,
              steps_per_epoch=FLAGS.eval_step_interval,
              epochs=training_epoch_max,
              validation_data=val_data,
              callbacks=[model_checkpoint_callback, tensorboard_callback])

    # Test and save the model.
    test_data = audio_processor.get_data(audio_processor.Modes.TESTING)
    test_data = test_data.batch(FLAGS.batch_size)

    # Evaluate the model performace.
    test_loss, test_acc = model.evaluate(x=test_data)
    print(f'Final test accuracy: {test_acc*100:.2f}%')

Invoke training procedure (**Warning:** This will take a very long time!)

In [ ]:
train()

Pick a checkpoint

In [6]:
FLAGS.checkpoint = ???  # TODO: pick best?

SyntaxError: invalid syntax (1662091861.py, line 1)

## 5. Test trained TensorFlow model

TODO

In [ ]:
test()

## 6. Quantization and Conversion to TFLite 

Define conversion procedure

In [8]:
def convert(model_settings, audio_processor, checkpoint, quantize, inference_type, tflite_path):
    """Load our trained floating point model and convert it.
    TFLite conversion or post training quantization is performed and the
    resulting model is saved as a TFLite file.
    We use samples from the validation set to do post training quantization.
    Args:
        model_settings: Dictionary of common model settings.
        audio_processor: Audio processor class object.
        checkpoint: Path to training checkpoint to load.
        quantize: Whether to quantize the model or convert to fp32 TFLite model.
        inference_type: Input/output type of the quantized model.
        tflite_path: Output TFLite file save path.
    """
    model = models.create_model(model_settings, False)
    model.load_weights(checkpoint).expect_partial()

    val_data = audio_processor.get_data(audio_processor.Modes.VALIDATION).batch(1)

    def _rep_dataset():  # TODO: make this a student task?
        """Generator function to produce representative dataset."""
        i = 0
        for mfcc, label in val_data:
            if i > NUM_REP_DATA_SAMPLES:
                break
            i += 1
            yield [mfcc]

    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    if quantize:
        # Quantize model and save to disk.
        if inference_type=='int8':
            converter.inference_input_type = tf.int8
            converter.inference_output_type = tf.int8

        # Int8 post training quantization needs representative dataset.
        converter.representative_dataset = _rep_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
    tflite_model = converter.convert()
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    print('{} model saved to {}.'.format("Quantized" if quantize else "Converted", tflite_path))

Prepare for conversion

In [ ]:
model_settings = models.prepare_model_settings(len(data.prepare_words_list(FLAGS.wanted_words.split(','))),
                                               FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
                                               FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)

audio_processor = data.AudioProcessor(data_url=FLAGS.data_url,
                                      data_dir=FLAGS.data_dir,
                                      silence_percentage=FLAGS.silence_percentage,
                                      unknown_percentage=FLAGS.unknown_percentage,
                                      wanted_words=FLAGS.wanted_words.split(','),
                                      validation_percentage=FLAGS.validation_percentage,
                                      testing_percentage=FLAGS.testing_percentage,
                                      model_settings=model_settings)

Invoke conversion

In [ ]:
tflite_path_quantized = f'{FLAGS.model_name}_quantized.tflite'
tflite_path = f'{FLAGS.model_name}.tflite'

# Load floating point model from checkpoint and convert it.
convert(model_settings, audio_processor, FLAGS.checkpoint,
        False, "fp32", tflite_path)

# Quantize model from checkpoint and convert it.
convert(model_settings, audio_processor, FLAGS.checkpoint,
        True, "int8", tflite_path_quantized)


## 7. Test Converted TFLite Model

Test the newly converted model on the test set.

**Floating Point**

In [ ]:
tflite_test(model_settings, audio_processor, tflite_path)

**Quantized**

In [ ]:
tflite_test(model_settings, audio_processor, tflite_path_quantized)

## 8. Visualize TFLite Model

TODO: Netron!

## 9. Performance and Memory Estimations

### 9.1 ROM Usage

TFLITE filesize: Weights+Graph-Metadata(Tensors/Operators) -> ? kB?

On-device:

Weights -> ROM (Float(4B)/Quantized(1B))

Graph -> Depends on inference engine
Kernel implementations -> ROM

* Alignment

In [11]:
### Task X: REPLACE CODE BELOW ###

### Task X: REPLACE CODE ABOVE ###

### 9.2 RAM Usage

Memory Planning!

Arena!

Worst case: sum of all intermedia buffers

inputs/outputs -> Depending on the "inference engine"

For linear/sequential models: biggest in/out pair of a layer

### 9.3 Number of MAC Operations

FLOPS vs MACS?

https://stackoverflow.com/questions/56138754/formula-to-compute-the-number-of-macs-in-a-convolutional-neural-network

https://leimao.github.io/blog/Depthwise-Separable-Convolution/ (pytorch)

https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec

https://cdmana.com/2021/04/20210413132349621b.html

https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d

(https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d)?

!!!https://machinethink.net/blog/how-fast-is-my-model/

### 9.4 Constraints fulfilled?

Example target: ESP32C3
    
Warning: drivers etc sensor stuff will contribute to equations as well

## Further considerations

Optimize for size?

Model compression: Pruning

Packing (Quantization-aware training)